#Resources

* https://www.youtube.com/watch?v=ZzgUqFtxgXI
* https://www.youtube.com/watch?v=7aBRk_JP-qY
* https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/3/document-splitting

#Introduction

Title: Exam MCQ Generator

Game Plan:

*   Divide the document into multiple chuncks and then select chunks at random, pass them to Langchain and ask to make an MCQ out of it

#Installs

In [1]:
!pip install -q python-dotenv

!pip install -q langchain_experimental
!pip install -q langchain

!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 re

#Imports

In [44]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

#Data Source

In [9]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
wizard_in_oz_text_path = '/content/drive/MyDrive/datasets/llm-from-scratch/wizard_in_oz.txt'

In [11]:
with open(wizard_in_oz_text_path, 'r', encoding='utf-8') as f:
  text = f.read()

In [12]:
print(text[:100])

﻿Title: Dorothy and the Wizard in Oz


Author: L. Frank Baum

Illustrator: John R. Neill

Release da


#Env

In [13]:
env_path = '/content/drive/MyDrive/credentials/data-analytics-demo/.env'

In [14]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(env_path)

True

In [17]:
GOOGLE_PALM_API_KEY = os.environ['GOOGLE_PALM_API_KEY']
HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']
OPEN_AI_API_KEY = os.environ['OPEN_AI_API_KEY']

# GOOGLE_PALM_API_KEY, HUGGINGFACE_API_KEY, OPEN_AI_API_KEY

#Model

In [46]:
langchain_llm = ChatOpenAI(openai_api_key=OPEN_AI_API_KEY, model_name="gpt-3.5-turbo-0613", verbose=False,)

In [22]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [39]:
texts = text_splitter.create_documents([text])

In [41]:
print(f'Total Documents Created: {len(texts)}')

Total Documents Created: 287


In [57]:
prompt = (
    "Text:"
    "\n{question}"
    "\nGenerate an Multiple Choice Question from this text"
    "\nDo not use any outside information"
)

In [58]:
prompt_template = ChatPromptTemplate.from_template(prompt)

prompt_template

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Text:\n{question}\nGenerate an Multiple Choice Question from this text\nDo not use any outside information'))])

In [59]:
llm_chain = LLMChain(llm=langchain_llm,prompt=prompt_template,output_key="result")

In [60]:
def get_question(doc_index):
  response = llm_chain.invoke({"question": texts[doc_index]})
  return response

#Testing

In [61]:
response = get_question(20)

In [62]:
print(response)

{'question': Document(page_content='blue and one orange. This splendid group of colored suns sent rays\ndarting in every direction, and as the horse and buggy--with Dorothy and\nZeb--sank steadily downward and came nearer to the lights, the rays\nbegan to take on all the delicate tintings of a rainbow, growing more\nand more distinct every moment until all the space was brilliantly\nilluminated.\nDorothy was too dazed to say much, but she watched one of Jim\'s big ears\nturn to violet and the other to rose, and wondered that his tail should\nbe yellow and his body striped with blue and orange like the stripes of\na zebra. Then she looked at Zeb, whose face was blue and whose hair was\npink, and gave a little laugh that sounded a bit nervous.\n"Isn\'t it funny?" she said.\nThe boy was startled and his eyes were big. Dorothy had a green streak\nthrough the center of her face where the blue and yellow lights came\ntogether, and her appearance seemed to add to his fright.\n"I--I don\'t s-s

In [63]:
print(response['result'])

What colors do Dorothy and Zeb's appearances change to in the brightly illuminated space?
a) Dorothy's face turns green and Zeb's face turns blue
b) Dorothy's face turns blue and Zeb's face turns pink
c) Dorothy's face turns yellow and Zeb's face turns orange
d) Dorothy's face turns violet and Zeb's face turns rose
